### PanStarrs

In [ ]:
from astropy.table import Table

In [ ]:
import matplotlib.pyplot as plt
import arya

In [ ]:
from astropy import units as u

In [ ]:
import numpy as np

In [ ]:
from pathlib import Path

In [ ]:
import sys
sys.path.append("..")
sys.path.append("../../imaging")

from phot_utils import xmatch, get_atm_extinction
from astropy.nddata import CCDData

In [ ]:
panstarrs = Table.read("../../survey_data/yasone2_panstarrs.fits")

In [ ]:
filename_g = "../yasone2/img_g_01/photometry.cat"
filename_r = "../yasone2/img_r_01/photometry.cat"
filename_i = "../yasone2/img_i_01/photometry.cat"


In [ ]:
get_zeropoint("i")

In [ ]:
import tomllib

In [ ]:
def get_zeropoint(filt):
    d = None
    with open(f"../std1/img_{filt}_11/flat_fielded-astrom-zeropoint.toml", "rb") as f:
        d = tomllib.load(f)

    return d["zeropoint"] + d["ap_corr"]

In [ ]:
def get_airmass(file):
    path = Path(file).parent
    img = CCDData.read(path / "flat_fielded.fits")
    return img.header["AIRMASS"]

In [ ]:
cat_g = Table.read(filename_g, hdu=2)
cat_r = Table.read(filename_r, hdu=2)
cat_i = Table.read(filename_i, hdu=2)

In [ ]:
panstarrs

In [ ]:
panstarrs_filt = (panstarrs["qualityFlag"] & 4).reshape(-1) > 0

for filt in ["g", "r", "i"]:
    panstarrs_filt &= panstarrs[f"{filt}Flags"].reshape(-1) & 2 == 0
    panstarrs_filt &= panstarrs[f"{filt}Flags"].reshape(-1) & 1 == 0
    # panstarrs_filt &= panstarrs[f"{filt}MeanPSFMagNpt"].reshape(-1) > 2
    


In [ ]:
panstarrs_good = panstarrs[panstarrs_filt]

In [ ]:
ext_g = get_atm_extinction(get_airmass(filename_g), "Sloan_g")[0]
ext_r = get_atm_extinction(get_airmass(filename_r), "Sloan_r")[0]
ext_i = get_atm_extinction(get_airmass(filename_i), "Sloan_i")[0]

ext_g, ext_r, ext_i

In [ ]:
ra0 = np.median(cat_r["ALPHA_J2000"])
dec0 = np.median(cat_r["DELTA_J2000"])

In [ ]:
plt.figure()
plt.scatter(cat_r["ALPHA_J2000"], cat_r["DELTA_J2000"], s=1, color="r")
plt.scatter(cat_g["ALPHA_J2000"], cat_g["DELTA_J2000"], s=0.8, color="g")
plt.scatter(cat_i["ALPHA_J2000"], cat_i["DELTA_J2000"], s=0.6, color="y")

plt.scatter(panstarrs["ra"], panstarrs["dec"], s=0.2)
plt.xlim(ra0 - 0.1, ra0+0.1)
plt.ylim(dec0 - 0.1, dec0 + 0.1)

In [ ]:
xmatch_idx, xmatch_dist, xmatch_filt_panstarrs = xmatch(cat_r["ALPHA_J2000"], cat_r["DELTA_J2000"], panstarrs_good["ra"].reshape(-1), panstarrs_good["dec"].reshape(-1), 1*u.arcsec)

In [ ]:
xmatch_g_idx, xmatch_g_dist, xmatch_g_filt = xmatch(cat_r["ALPHA_J2000"], cat_r["DELTA_J2000"], cat_g["ALPHA_J2000"], cat_g["DELTA_J2000"], 1*u.arcsec)
xmatch_i_idx, xmatch_i_dist, xmatch_i_filt = xmatch(cat_r["ALPHA_J2000"], cat_r["DELTA_J2000"], cat_i["ALPHA_J2000"], cat_i["DELTA_J2000"], 1*u.arcsec)

In [ ]:
xmatch_filt = xmatch_filt_panstarrs & xmatch_g_filt & xmatch_i_filt

In [ ]:
panstarrs_xmatch = panstarrs_good[xmatch_idx]
osiris_x_panstarrs = cat_r[xmatch_filt]

In [ ]:
cat = cat_r.copy()
idx_aper = 5
cat["mag_g"] = cat_g["MAG_APER"][xmatch_g_idx, idx_aper] + get_zeropoint("g")  + 2.5 * np.log10(190) - ext_g - 2.5 * np.log10(1.9)
cat["mag_i"] = cat_i["MAG_APER"][xmatch_i_idx, idx_aper] + get_zeropoint("i")  + 2.5 * np.log10(190)  - ext_i - 2.5 * np.log10(1.9)
cat["mag_r"] = cat["MAG_APER"][:, idx_aper] + get_zeropoint("r")  + 2.5 * np.log10(190)  - ext_r - 2.5 * np.log10(1.9)

for filt in ["g", "r", "i"]:
    cat[f"mag_{filt}_panstarrs"] = panstarrs_xmatch[f"{filt}MeanApMag"].reshape(-1)
    cat[f"mag_{filt}_err_panstarrs"] = panstarrs_xmatch[f"{filt}MeanApMagErr"].reshape(-1)

    cat[f"mag_{filt}_panstarrs"][~xmatch_filt_panstarrs] = np.nan
    cat[f"mag_{filt}_err_panstarrs"][~xmatch_filt_panstarrs] = np.nan
        

In [ ]:
plt.figure()
plt.scatter(cat["ALPHA_J2000"]/u.degree, cat["DELTA_J2000"]/u.degree, s=0.3)

plt.scatter(panstarrs_xmatch["ra"], panstarrs_xmatch["dec"])
plt.scatter(cat["ALPHA_J2000"][xmatch_filt]/u.degree, cat["DELTA_J2000"][xmatch_filt]/u.degree, s=1)
# plt.xlim(-0.1, 0.1)
# plt.ylim(-0.1, 0.1)

In [ ]:
panstarrs

In [ ]:

fig, axs = plt.subplots(2, 3, sharex=True, sharey="row", figsize=(7, 4))

for i in range(3):
    plt.sca(axs[0][i])

    filt = ["g", "r", "i"][i]
    mag0 = cat[f"mag_{filt}"]
    mag_ps = cat[f"mag_{filt}_panstarrs"]
    mag_ps_err = cat[f"mag_{filt}_err_panstarrs"]
    plt.scatter(mag0, mag_ps, s=1, c=np.log2(cat["FLAGS"].data + 1))
    plt.xlim(26, 15)
    plt.ylim(26, 15)
    plt.plot([26, 10], [26, 10], color="black")
    if i == 0:
        plt.ylabel(f"{filt} (PS)")
    
    plt.sca(axs[1][i])
    
    yerr = np.maximum(mag_ps_err, 0)
    plt.errorbar(mag0[1:-1:20], (mag_ps-mag0)[1:-1:20], yerr=yerr[1:-1:20], fmt=".", capsize=0, )
    plt.scatter(mag0, mag_ps - mag0, s=1, c=np.log2(cat["FLAGS"].data + 1))

    plt.ylim(3, -3)
    plt.axhline(0, color="black")
    
    plt.xlabel(f"{filt} (osiris)")

plt.tight_layout()

# Catalogue creation